[eosio.token.cpp](https://github.com/EOSIO/eosio.contracts/blob/c046863a65d7e98424312ee8009f0acb493e6231/contracts/eosio.token/src/eosio.token.cpp#L87)

```C++
void token::transfer( name    from,
                      name    to,
                      asset   quantity,
                      string  memo )
{
    check( from != to, "cannot transfer to self" );
    require_auth( from );
    check( is_account( to ), "to account does not exist");
    auto sym = quantity.symbol.code();
    stats statstable( _self, sym.raw() );
    const auto& st = statstable.get( sym.raw() );

    require_recipient( from );
    require_recipient( to );

    check( quantity.is_valid(), "invalid quantity" );
    check( quantity.amount > 0, "must transfer positive quantity" );
    check( quantity.symbol == st.supply.symbol, "symbol precision mismatch" );
    check( memo.size() <= 256, "memo has more than 256 bytes" );

    auto payer = has_auth( to ) ? to : from;

    sub_balance( from, quantity );
    add_balance( to, quantity, payer );
}
```

In [1]:
import init
from uuoskit import uuosapi
from uuoskit import wasmcompiler
init.init()
test_account1 = init.test_account1
test_account2 = init.test_account2

In [2]:
code = r'''
package main

import "github.com/uuosio/chain"
import "github.com/uuosio/chain/logger"

type Transfer struct {
    From     chain.Name
    To       chain.Name
    Quantity chain.Asset
    Memo     string
}

func main() {
    _, firstReceiver, action := chain.GetApplyArgs()
    if firstReceiver == chain.NewName("eosio.token") && action == chain.NewName("transfer") {
        data := chain.ReadActionData()
        t := Transfer{}
        t.Unpack(data)
        if t.To == chain.NewName("eosio") && t.Quantity.Symbol == chain.NewSymbol("EOS", 4) {
            logger.Println("Example1, memo:", t.Memo)
        }
    }
}
'''

code, abi = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [3]:
r = uuosapi.transfer(test_account1, "eosio", 0.1, 'hello,world')
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['action_traces'][0]['inline_traces'][0]['console'])
print(r['processed']['action_traces'][0]['inline_traces'][1]['console'])
print(r['processed']['elapsed'])



Example1, memo: hello,world


665


## Improved code

In [5]:
code = r'''
package main

import (
    "github.com/uuosio/chain"
    "github.com/uuosio/chain/logger"
)

//contract hello
type MyContract struct {
    Receiver      chain.Name
    FirstReceiver chain.Name
    Action        chain.Name
}

func NewContract(receiver, firstReceiver, action chain.Name) *MyContract {
    return &MyContract{receiver, firstReceiver, action}
}

//notify transfer
func (c *MyContract) Transfer(from, to chain.Name, quantity chain.Asset, memo string) {
    if c.FirstReceiver == chain.NewName("eosio.token") && c.Action == chain.NewName("transfer") {
        if to == chain.NewName("helloworld11") && quantity.Symbol == chain.NewSymbol("EOS", 4) {
            logger.Println("Example2, memo:", memo)
        }
    }
}
'''

code, abi = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [6]:
r = uuosapi.transfer(test_account2, test_account1, 0.1, 'goodbye,world')
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['action_traces'][0]['inline_traces'][0]['console'])
print(r['processed']['action_traces'][0]['inline_traces'][1]['console'])
print(r['processed']['elapsed'])


helloworld12 eosio.token transfer
goodbye,world

Example2, memo: goodbye,world

694
